In [1]:
!nvidia-smi

Thu Jun  5 03:47:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install transformers torchtext datasets fsspec==2023.6.0

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidi

In [3]:
import torch
from transformers import BertTokenizer, BertModel
from datasets import load_dataset
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
!rm -rf ~/.cache/huggingface/datasets/imdb
!rm -rf ~/.cache/huggingface/modules

In [5]:
main_dataset = load_dataset('imdb', revision="main")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [20]:
import torch
from transformers import BertTokenizer, BertModel
from datasets import load_dataset
import gc
from torch.utils.data import DataLoader

class MultiHeadAttention(torch.nn.Module):
    def __init__(self, input_dim, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_k = input_dim // num_heads

        self.Q = torch.nn.Linear(input_dim, input_dim)
        self.K = torch.nn.Linear(input_dim, input_dim)
        self.V = torch.nn.Linear(input_dim, input_dim)

        self.fc = torch.nn.Linear(input_dim, input_dim)

    def forward(self, x, mask):
        # Implement the forward pass
        # x should be of shape (batch_size, seq_length, input_dim)
        batch_size, seq_length, _ = x.shape
        Q = self.Q(x).view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        K = self.K(x).view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        V = self.V(x).view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

        # new format: (batch_size, num_heads, seq_length, d_k)

        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / (K.size(-1) ** 0.5)

        if mask is not None:
            # Apply the mask to the attention scores
            mask = mask.unsqueeze(1).unsqueeze(2)
            attention_scores = attention_scores.masked_fill(mask == 0, float('-inf'))

        # format: (batch_size, num_heads, seq_length, seq_length)

        attention_weights = torch.nn.functional.softmax(attention_scores, dim=-1)

        context = torch.matmul(attention_weights, V)

        # format: (batch_size, num_heads, seq_length, d_k)

        context = context.transpose(1, 2).contiguous().view(batch_size, seq_length, -1)

        # format: (batch_size, seq_length, input_dim)

        # after concatnating the heads we run a linear layer to allow the different parts of the embeddings to interact post attention

        output = self.fc(context)
        return output


class FeedForwardNetwork(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super(FeedForwardNetwork, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, input_dim)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


class SentimentAnalysisModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=1):
        super(SentimentAnalysisModel, self).__init__()

        self.multi_head_attention = MultiHeadAttention(input_dim)
        self.norm1 = torch.nn.LayerNorm(input_dim)
        self.dropout1 = torch.nn.Dropout(0.1)
        self.feed_forward = FeedForwardNetwork(input_dim, hidden_dim)
        self.norm2 = torch.nn.LayerNorm(input_dim)
        self.dropout2 = torch.nn.Dropout(0.1)

        self.multi_head_attention2 = MultiHeadAttention(input_dim)
        self.norm3 = torch.nn.LayerNorm(input_dim)
        self.dropout3 = torch.nn.Dropout(0.1)
        self.feed_forward2 = FeedForwardNetwork(input_dim, hidden_dim)
        self.norm4 = torch.nn.LayerNorm(input_dim)
        self.dropout4 = torch.nn.Dropout(0.1)

        # self.output_layer = torch.nn.Linear(input_dim, output_dim)
        # self.dropout5 = torch.nn.Dropout(0.1)
        # self.sigmoid = torch.nn.Sigmoid()

        # Output layers
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(hidden_dim, output_dim)
        )

        self.optimizer = torch.optim.Adam(self.parameters(),  lr=2e-5, weight_decay=1e-4)


    def forward(self, x, mask):
        # x should be of shape (batch_size, seq_length, input_dim)
        residual = x
        x = self.multi_head_attention(x, mask)
        x = self.dropout1(x)
        x = self.norm1(x + residual)

        residual = x
        x = self.feed_forward(x)
        x = self.dropout2(x)
        x = self.norm2(x + residual)

        residual = x
        x = self.multi_head_attention2(x, mask)
        x = self.dropout3(x)
        x = self.norm3(x + residual)

        residual = x
        x = self.feed_forward2(x)
        x = self.dropout4(x)
        x = self.norm4(x + residual)

        mask_expanded = mask.unsqueeze(-1).float()
        x = x * mask_expanded
        pooled = x.sum(dim=1) / mask.sum(dim=1, keepdim=True).clamp(min=1e-9)

        # x = x.mean(dim=1)

        # x = self.output_layer(pooled)
        # x = self.dropout5(x)
        # x = self.sigmoid(x)

        x = self.classifier(pooled).squeeze(-1)

        return x

    def compute_loss(self, logits, targets):
        loss_fn =  torch.nn.BCEWithLogitsLoss()
        targets = targets.float()
        return loss_fn(logits.squeeze(), targets)

    def train_step(self, inputs, mask, targets):
      self.train()
      self.optimizer.zero_grad()

      # Forward pass
      logits = self(inputs, mask)

      # Compute loss
      loss = self.compute_loss(logits, targets)

      # Backward pass and optimize
      loss.backward()
      torch.nn.utils.clip_grad_norm_(self.parameters(), 1.0)
      self.optimizer.step()

      return loss.item()

    def test_model(self, inputs, mask, targets):
        self.eval()
        total_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        with torch.no_grad():
          # Remove extra dimensions if present
          inputs = inputs.squeeze(0) if inputs.dim() > 3 else inputs
          mask = mask.squeeze(0) if mask.dim() > 2 else mask
          targets = targets.squeeze(0) if targets.dim() > 1 else targets

          # Forward pass
          logits = self(inputs, mask)
          logits = torch.sigmoid(logits)

          # Compute loss
          loss = self.compute_loss(logits, targets)
          total_loss += loss.item()

          # Compute accuracy
          predictions = (logits.squeeze() > 0.5).float()
          correct_predictions += (predictions == targets.float()).sum().item()
          total_predictions += targets.size(0)

        avg_loss = total_loss / inputs.shape[0]
        accuracy = correct_predictions / total_predictions

        # print("Predicted: ", logits)
        # print("Actual: ", targets)
        print(avg_loss, accuracy)

        return avg_loss, accuracy

class Databuilder:
    def __init__(self, batch_size=32):
        self.dataset = main_dataset
        self.train_data = self.dataset['train'].shuffle(seed=42)
        self.test_data = self.dataset['test'].shuffle(seed=42)
        self.batch_size = batch_size
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.embedder = BertModel.from_pretrained('bert-base-uncased').to(device)

        for param in self.embedder.parameters():
            param.requires_grad = False

    def get_loader(self, type='train'):
      batch_texts = []
      batch_labels = []

      loaded_data = self.dataset[type].shuffle()
      for example in loaded_data:

          batch_texts.append(example['text'])
          batch_labels.append(int(example['label']))

          if len(batch_texts) == self.batch_size:
              yield self.vectorize_batch(batch_texts, batch_labels)

              gc.collect()

              batch_texts = []
              batch_labels = []

    def vectorize_batch(self, batch_texts, batch_labels):
      tokens = self.tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt', max_length=512)

      input_ids = tokens['input_ids'].to(device)
      attention_mask = tokens['attention_mask'].to(device)

      with torch.no_grad():
        embeddings = self.embedder(input_ids).last_hidden_state

      embeddings = embeddings.detach()

      labels = torch.tensor(batch_labels, dtype=torch.int8)

      return (embeddings, attention_mask, labels)



In [ ]:
print("Loading dataset...")
epochs = 10

databuilder = Databuilder(batch_size=128)

transformer = SentimentAnalysisModel(input_dim=768, hidden_dim=2048, output_dim=1).to(device)

test_batch = next(databuilder.get_loader('test'))

ti, tm, tt = test_batch
ti = ti.to(device)
tm = tm.to(device)
tt = tt.to(device)

for epoch in range(epochs):
  print(epoch)
  for batch in databuilder.get_loader('train'):
    inputs, masks, targets = batch

    inputs = inputs.to(device)
    masks = masks.to(device)
    targets = targets.to(device)

    loss = transformer.train_step(inputs, masks, targets)

    loss, acc = transformer.test_model(ti, tm, tt)

    # if acc >= 0.9:
    #   break

    gc.collect()

print("Training complete.")

In [8]:
# Cleanup
gc.collect()
torch.cuda.empty_cache()

In [24]:
# Test

test_batches = ['The movie was bad', "I want my money back!", "OMG I WAS SCREAMING, IT WAS THAT GOOD!", 'AMAZING MOVIE!']

# databuilder = Databuilder(batch_size=64)

# transformer = SentimentAnalysisModel(input_dim=768, hidden_dim=2048, output_dim=1).to(device)

i, m, l = databuilder.vectorize_batch(test_batches, [])

# print(i,m,l)

print(torch.sigmoid(transformer.forward(i, m)))

for text, sentiment in zip(test_batches, torch.sigmoid(transformer.forward(i, m))):
  sentiment = "Good" if sentiment > 0.5 else "Bad"
  print(text, "=>", sentiment)



tensor([0.0442, 0.3883, 0.3975, 0.8679], device='cuda:0',
       grad_fn=<SigmoidBackward0>)
The movie was bad => Bad
I want my money back! => Bad
OMG I WAS SCREAMING, IT WAS THAT GOOD! => Bad
AMAZING MOVIE! => Good
